In [1]:
import os
import sys
import pandas as pd
import childespy
import numpy as np
from os.path import join, exists

R[write to console]: Installing package into ‘/home/stephan/R/x86_64-pc-linux-gnu-library/4.1’
(as ‘lib’ is unspecified)



Reinstalling childesr version 0.2.1


R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/childesr_0.2.3.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 22865 bytes (22 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to conso

In [2]:
sys.path.append('../../')
from src.utils import split_gen, data_cleaning, load_splits, configuration, paths
config = configuration.Config()
np.random.seed(config.SEED)

# Generate Train, Val for All and Age splits

In [ ]:
#get all of the North American and British English adult and child utterances without xxx or yyy
#concatenate them at the the transcript level
#hold out 20% for validation 

In [3]:
corpora = childespy.get_sql_query('select * from corpus where \
collection_name in ("Eng-NA", "Eng-UK") and data_source = "CHILDES"')
corpora

R[write to console]: Using current database version: '2021.1'.



,id,name,collection_name,data_source,collection_id
1,34,Garvey,Eng-NA,CHILDES,2
2,35,Valian,Eng-NA,CHILDES,2
3,36,Bernstein,Eng-NA,CHILDES,2
4,37,Clark,Eng-NA,CHILDES,2
5,38,PetersonMcCabe,Eng-NA,CHILDES,2
...,...,...,...,...,...
59,229,Wells,Eng-UK,CHILDES,12
60,230,Gathburn,Eng-UK,CHILDES,12
61,231,Nuffield,Eng-UK,CHILDES,12
62,232,Lara,Eng-UK,CHILDES,12


In [4]:
childes_datasets = ",".join([str(x) for x in corpora.collection_id])

In [5]:
save_path = join(config.finetune_dir, 'utt_glosses.pkl')

if False:
#if config.regenerate:
    utt_glosses = childespy.get_sql_query('select gloss, transcript_id, id, \
    utterance_order, speaker_code, target_child_name, corpus_name, target_child_age, type from utterance where collection_name in ("Eng-NA", "Eng-UK") \
    and collection_id in ('+childes_datasets+') and speaker_code in ("MOT", "FAT","CHI")' , db_version = "2020.1")
    utt_glosses.to_pickle(save_path)
else: 
    utt_glosses = pd.read_pickle(save_path)

In [6]:
utt_glosses = utt_glosses[utt_glosses.gloss != ''] # Added 8/16/21 -- drop empty utterances

all_utt_glosses = utt_glosses.rename(columns = {'id' : 'utterance_id'}).copy()
# end both cites

## "all" split

In [48]:
import imp
imp.reload(paths)
imp.reload(data_cleaning)
imp.reload(split_gen)

<module 'src.utils.split_gen' from '../../src/utils/split_gen.py'>

In [ ]:
# expectations for verbose output.
# 232 should be (4205071, 7) (because you added two extra fields)
# 233 likewise is (3959952, 7)

# Save the partition markings as well, for verification purposes.

In [9]:
success_sample_path = paths.get_sample_csv_path('training', 'Providence', 'all', 'success', age = None, n=5000)

In [10]:
success_sample_path

'/home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/sample/n=5000/Providence_all/training_success_utts_5000.csv'

### No tags

In [40]:
data = data_cleaning.drop_errors(all_utt_glosses)
data_no_tags = data_cleaning.clean_glosses_and_add_speaker_label(data, use_tags = False)
assert not any(['xxx' in s for s in set(data_no_tags.gloss_with_punct)])
assert not any(['yyy' in s for s in set(data_no_tags.gloss_with_punct)])
data_no_tags['tokens'] = [str(x).lower().split(' ') for x in data_no_tags.gloss]

In [41]:
Providence_notags_split_folder = paths.get_directory({
    "task_phase" : 'extract_data',
    "training_split" : 'Providence',
    "training_dataset" : 'all',    
    "test_split" : None,
    "test_dataset" : None,
    "model_type" : None,
    "context_width" : None,
    "n_samples" : config.n_across_time,
    "task_name" : None,
    "use_tags" : False
})
Providence_notags_split_folder

'/home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence_all_no_tags'

In [56]:
notags_split_glosses_df, notags_train_df = split_gen.exec_split_gen(
    data_no_tags, Providence_notags_split_folder, 'train')

File written to /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence_all_no_tags/train.txt
File written to /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence_all_no_tags/eval.txt
Writing split glosses to: /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence_all_no_tags/data_pool_with_phases.pkl


### With tags

In [44]:
data = data_cleaning.drop_errors(all_utt_glosses)
data_with_tags = data_cleaning.clean_glosses_and_add_speaker_label(data, use_tags = True)
assert not any(['xxx' in s for s in set(data_with_tags.gloss_with_punct)])
assert not any(['yyy' in s for s in set(data_with_tags.gloss_with_punct)])
data_with_tags['tokens'] = [str(x).lower().split(' ') for x in data_with_tags.gloss]

/home/stephan/notebooks/nicole/child-directed-listening/cdl_env/lib/python3.6/site-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
../../src/utils/data_cleaning.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  for x in data.type ]
../../src/utils/data_cleaning.py:195: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

In [49]:
Providence_withtags_split_folder = paths.get_directory({
    "task_phase" : 'extract_data',
    "training_split" : 'Providence',
    "training_dataset" : 'all',    
    "test_split" : None,
    "test_dataset" : None,
    "model_type" : None,
    "context_width" : None,
    "n_samples" : config.n_across_time,
    "task_name" : None,
    "use_tags" : True
})
Providence_withtags_split_folder

'/home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence_all_with_tags'

In [55]:
withtags_split_glosses_df, withtags_train_df = split_gen.exec_split_gen(
    data_with_tags, Providence_withtags_split_folder, 'train')

../../src/utils/data_cleaning.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utt_data['contains_error'] = ['xxx' in str(x) or 'yyy' in str(x) for x in all_lowercase]


File written to /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence_all_with_tags/train.txt
File written to /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence_all_with_tags/eval.txt
Writing split glosses to: /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence_all_with_tags/data_pool_with_phases.pkl


## "age" split

In [57]:
for use_tags in (True, False):
    
    if use_tags: 
        split_glosses_df = withtags_split_glosses_df
    else:
        split_glosses_df = notags_split_glosses_df
    
    young_df, old_df = split_gen.get_age_split_data(split_glosses_df.copy(), months = config.age_split)

    for args, which_df in zip([('Providence-Age', 'old'), ('Providence-Age', 'young')],
            [old_df, young_df]):

        split, dataset = args
        this_split_folder = paths.get_directory({
                "task_phase" : 'extract_data',
                "training_split" : split,
                "training_dataset" : dataset,    
                "test_split" : None,
                "test_dataset" : None,
                "model_type" : None,
                "context_width" : None,
                "n_samples" : config.n_across_time,
                "task_name" : None,
                "use_tags" : use_tags
            })
        
        split_gen.exec_split_gen(which_df, this_split_folder, 'train')    

../../src/utils/data_cleaning.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utt_data['contains_error'] = ['xxx' in str(x) or 'yyy' in str(x) for x in all_lowercase]


File written to /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence-Age_old_with_tags/train.txt
File written to /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence-Age_old_with_tags/eval.txt
Writing split glosses to: /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence-Age_old_with_tags/data_pool_with_phases.pkl
File written to /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence-Age_young_with_tags/train.txt
File written to /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence-Age_young_with_tags/eval.txt
Writing split glosses to: /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence-Age_young_with_tags/data_pool_with_phases.pkl


../../src/utils/data_cleaning.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  utt_data['contains_error'] = ['xxx' in str(x) or 'yyy' in str(x) for x in all_lowercase]


File written to /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence-Age_old_no_tags/train.txt
File written to /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence-Age_old_no_tags/eval.txt
Writing split glosses to: /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence-Age_old_no_tags/data_pool_with_phases.pkl
File written to /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence-Age_young_no_tags/train.txt
File written to /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence-Age_young_no_tags/eval.txt
Writing split glosses to: /home/stephan/notebooks/child-directed-listening/output/experiments/full_scale/extract_data/n=5000/Providence-Age_young_no_tags/data_pool_with_phases.pkl


In [60]:
from datetime import datetime
print(datetime.today())

2022-04-20 12:20:38.767035
